In [ ]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup as bs
from fuzzywuzzy import process,fuzz
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score,mean_absolute_error,r2_score

data= get("https://archive.ics.uci.edu/ml/machine-learning-databases/movies-mld/data/main.html")

# Kaggle DataSet ="https://www.kaggle.com/PromptCloudHQ/imdb-data/data"
kaggle= pd.read_csv("IMDB-Movie-Data.csv")

In [ ]:
soup= bs(data.content,"lxml")
table = soup.findAll('table')
movieslist=[]
for i in table:
    x=i.findAll('tr')
    for y in x:
        try:
            movieslist.append( str( y.findAll('td')[1] ).split(":")[1].split("<")[0] )
        except:
            pass
#print(movieslist)

In [ ]:
#Kaggle Data Feature Preparation
mean_Revenue=kaggle["Revenue (Millions)"][kaggle["Revenue (Millions)"].notnull()].mean()
kaggle["Revenue (Millions)"].fillna(mean_Revenue,inplace=True)

mean_Metascore = kaggle["Metascore"][kaggle["Metascore"].notnull()].mean()
kaggle["Metascore"].fillna(mean_Metascore,inplace=True)

features=["Votes","Revenue (Millions)","Metascore","Year","Runtime (Minutes)"]
XX=kaggle[features]

y="Rating"
Y=kaggle["Rating"]
movies = kaggle["Title"]

scaler = MinMaxScaler()
scaler.fit(kaggle[["Votes","Revenue (Millions)","Metascore"]])
X=pd.DataFrame(scaler.transform(kaggle[["Votes","Revenue (Millions)","Metascore"]]))
X["Year"],X["Runtime (Minutes)"]=kaggle["Year"],kaggle["Runtime (Minutes)"]
X.columns =(features)
#print(X.isnull().values.any())
#print(X.head())

# SVM Model

In [ ]:
#svm Model
from sklearn import svm
clf=svm.SVR()
clf.fit(X,Y)
predicted=clf.predict(X)
joblib.dump(clf,"svm.pkl")
clf = joblib.load('svm.pkl') 
predicted=clf.predict(X)
#print(X.shape)
#print((abs(predicted)-abs(Y) ).mean())

In [ ]:
def movie_rating_SVM(movieName="Step UP"):
    i=process.extractOne(movieName,movies ,scorer=fuzz.token_sort_ratio)
    Xtest=kaggle.iloc[i[2]][features]
    clf = joblib.load('svm.pkl')
    Ypredicted=clf.predict(Xtest.values.reshape(1,5))
    Ytrue=kaggle.iloc[i[2]][y]
    print("SVM",{"mean_absolute_error":mean_absolute_error([Ytrue],Ypredicted),
          "r2_scor": r2_score([Ytrue],Ypredicted)})
    return("%.2f/%.2f "  %(Ypredicted[0] ,Ytrue))

# Lasso Lars Regression

In [ ]:
# LassoLars
from sklearn import linear_model
reg = linear_model.LassoLars(alpha=0,normalize=False)
(reg.fit(X,Y))
joblib.dump(reg,"lasso.pkl")
clf = joblib.load('lasso.pkl') 
predicted=clf.predict(X)
#print(clf.coef_)
#print((abs(predicted)-abs(Y)).mean())

In [ ]:
def movie_rating_Lasso(movieName="Step UP"):
    i=process.extractOne(movieName,movies ,scorer=fuzz.token_sort_ratio)
    Xtest=X.iloc[i[2]][features]
    clf = joblib.load('lasso.pkl')
    Ypredicted=clf.predict(Xtest.values.reshape(1,5))
    Ytrue= kaggle.iloc[i[2]][y]
    print("Lasso",{"mean_absolute_error":mean_absolute_error([Ytrue],Ypredicted),
          "r2_scor": r2_score([Ytrue],Ypredicted)})
    return("%.2f/%.2f "  %(Ypredicted[0] ,Ytrue))


# Model Comparison

In [ ]:
# User ->To Enter the Movie Name as Argument
# Example -> movie_rating_SVM("Step UP")

print("SVM:",movie_rating_SVM())
print("Lasso:",movie_rating_Lasso())

Interpretation => On removing last two features while training SVM model, it will give better accuracy.

In [ ]:
# Futher Hypertuning of Parameters

#from sklearn.model_selection import GridSearchCV
#clf=svm.SVR()
#params =dict(kernal=['linear', 'rbf', 'sigmoid'],)
#grid = GridSearchCV(clf,params,cv=10,scoring="accuracy",n_jobs=-1)
#grid.fit(X,Y)

